## Test of accelerated Monte Carlo ##

In [1]:
import sys
sys.path.insert(0, "../python/")

from particle import particle
from physics import em_physics
from cylinder import cylinder

import matplotlib.pyplot as plt
import numpy as np
from numba import jit
import pandas as pd

import json
import uproot

### Define the Geometry ###

In [2]:
# define the geometry
radius = 50 # cm
height = 100 #cm
cryostat = cylinder(R=radius,h=height)

radius = 40 # cm
height = 80 #cm
fiducial = cylinder(R=radius,h=height)

cylinder::__init__ Define cylinder with R= 50  and height= 100
cylinder::__init__ Define cylinder with R= 40  and height= 80


### Setup the photon physics ###

In [3]:
# define the physics
em = em_physics()

### Event generation ###

In [ ]:
#
# number of events
#
nevent = 1000001
#
# gamma energy
#
energy = 1000 # keV
#
# output filename
#
#write_mode = 'json'
write_mode = 'txt'

mcout = '../mcdata/testdata' + '.' + write_mode 

#
# open output data file
#
f = open(mcout,'w')
if write_mode == 'json':
    f.write('[\n')    
    
#
# event generation loop
#
for ieve in range(nevent):
    
    if ieve%25000 == 0:
        print("generated ",ieve," events")
    #
    # make a particle
    #
    p = particle(type='gamma',
                 energy=energy, 
                 geometry=cryostat, 
                 fiducial=fiducial, 
                 vrt='fiducial_scatter',
                 edep_max=250,
                 nscatter_max=2,
                 physics=em,
                 debug=False)
    
    #
    # propagate the particle and retrieve the intersection points
    #
    p.propagate()

    #
    # store data
    #
    if write_mode == 'txt':
        wstring = "{:d} {:d} {:7.6e} {:7.6f} {:7.6f} {:7.6f} {:7.6f} \n".format(ieve,len(p.xint), 
                                                                                p.weight,p.edep,
                                                                                p.x0start[0],p.x0start[1],p.x0start[2])
        f.write(wstring)
    
    for i in range(len(p.xint)):
        #
        # data structure for each energy deposit
        #
        if write_mode == 'json':
            dd = {'x':p.xint[i][0][0],'y':p.xint[i][0][1],'z':p.xint[i][0][2],'de':p.xint[i][1],
                  'w':p.weight,'n':p.nscatter}
            json.dump(dd,f, sort_keys=True, indent=8)
            f.write(',\n')
        else:
            wstring = "{:7.6f} {:7.6f} {:7.6f} {:7.6f} \n".format(p.xint[i][0][0],
                                                                  p.xint[i][0][1],
                                                                  p.xint[i][0][2],
                                                                  p.xint[i][1])
            f.write(wstring)
             
#
# close output data file
#
if write_mode == 'json':
    f.write('{}]')
else:
    f.write('-1\n')
    
f.close()

generated  0  events
generated  25000  events
generated  50000  events
